In [0]:
#Imports
import keras
import keras.layers
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import random
import copy
import tensorflow as tf
from tensorflow import set_random_seed


#Presets and things thatneed to be run first
num_images = 10000
num_generations = 25
max_num_dense_nodes = 400
random.seed(1000)
set_random_seed(2)
#Bad seeds: 0
#Good seeds: 

#Import data and split into one large test set and multiple training and validation sets
data = pd.read_csv('train.csv')
images = data.iloc[:,1:]
images = images / 255.0
images = images.values.reshape(42000, 28, 28, 1)
labels = data.iloc[:,:1].values.ravel()

train_images, test_images, train_labels, test_labels = train_test_split(images, labels, train_size = 20000, random_state=1)

train_data = []
for i in range(10):
    train_data.append(train_test_split(train_images, train_labels, train_size = 2000, random_state=i))



Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [0]:
#LayerMetadata and derived classes—these classes wrap metadata so that information
#about models can be moved around without returning an entire model
class LayerMetadata:
    def __init__(self, data_size):
        self.data = np.zeros(data_size, dtype="int")
        
    def to_string(self):
        return "badd oop"

class ConvLayerMetadata(LayerMetadata):
    def __init__(self, kernels, pool_type, padding):
        LayerMetadata.__init__(self, 3)
        self.data[0] = max(1, kernels)
        self.data[1] = pool_type #0 = MaxPooling, 1=AveragePooling
        self.data[2] = padding #0 = valid, 1=same
        
    def to_string(self):
        return "Conv " + str(self.data[0]) + " "
        
class DenseLayerMetadata(LayerMetadata):
    def __init__(self, nodes):
        LayerMetadata.__init__(self, 1)
        self.data[0] = max(10, nodes)
        
    def to_string(self):
        return "Dense " + str(self.data[0]) + " "

#A class containing a list of LayerMetadata and its evolvedness
class ModelMetadata:
    def __init__(self, layers, evolvedness, accuracy, data_number):
        self.layers = layers
        self.evolvedness = evolvedness
        self.accuracy = accuracy
        self.data_number = data_number
        
    def to_string(self):
        str = ""
        for layer in self.layers:
            str += layer.to_string()
        e = repr(self.evolvedness)
        a = repr(self.accuracy)
        str += " Evolvedness: " + e + " Accuracy:" + a
        return str

#A class comprising a Keras model and metadata about it, along with static
#methods useful for building new ModelWithMetadatas
#When constructed with no arguments, a random model is generated, otherwise the
#model is constructed from the input metadata
class ModelWithMetadata:
    def __init__(self, metadata):
        if metadata == None:
            self.metadata = self.get_random_metadata()
            self.model = self.get_from_metadata(self.metadata)
        else:
            self.model = self.get_from_metadata(metadata)
            self.metadata = metadata
        self.metadata.accuracy = self.get_accuracy()
        
    #Gets the accuracy of a model
    def get_accuracy(self):
        ada = tf.train.AdamOptimizer()
        try:
            self.model.compile(optimizer=ada, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
            x = self.metadata.data_number
            self.model.fit(train_data[x][0], train_data[x][2], batch_size=500, epochs=20, verbose=0,)
            return self.model.evaluate(train_data[x][1], train_data[x][3], verbose=0)[1]
        except Exception as e:
            print(e)
            print("Model failed")
            return 0.0
    
    #Returns a model determined by input metadata
    @staticmethod
    def get_from_metadata(metadata):
        model = keras.Sequential()
        try: 
            added_dense_layer = False
            added_first_layer = False
            for layer in metadata.layers:
                if type(layer) is ConvLayerMetadata:
                    if not added_first_layer:  
                        model.add(keras.layers.Conv2D(layer.data[0], 2, strides=(1,1),input_shape=(28, 28, 1),activation="relu", padding= "valid" if layer.data[2] == 0 else "same"))
                        added_first_layer = True
                    else:
                        model.add(keras.layers.Conv2D(layer.data[0], 2, strides=(1,1),activation="relu",padding = "valid"))
                    if layer.data[1] == 0:
                         model.add(keras.layers.MaxPooling2D(pool_size=2, strides=None, padding= "valid" if layer.data[2] == 0 else "same"))
                    else:
                         model.add(keras.layers.AveragePooling2D(pool_size=2, strides=None, padding= "same" if layer.data[2] == 1 else "valid"))
                if type(layer) is DenseLayerMetadata:
                    if not added_dense_layer:
                        model.add(keras.layers.Flatten())
                        added_dense_layer = True
                    model.add(keras.layers.Dense(layer.data[0], activation="relu", use_bias=True, kernel_initializer="glorot_uniform", bias_initializer="zeros"))
            model.add(keras.layers.Dense(10, activation="softmax", use_bias=True, kernel_initializer="glorot_uniform", bias_initializer="zeros"))
            return model
        except Exception as e:
            print(e)
            #This shouldn't happen and is a bug. However, it doesn't impact
            #program performance in a non-negligible way, so it represents a
            #longterm TODO
            
    
    #Returns a ModelMetadata determined randomly
    @staticmethod
    def get_random_metadata():
        layers = []
        #for i in range(random.randint(1,3)):
            #kernels = random.randint(1, 150)
            #pool_type = random.randint(0,1)
            #padding = random.randint(0,1)
            #layers.append(ConvLayerMetadata(kernels, pool_type, padding))
        for i in range(random.randint(3,6)):
            layers.append(DenseLayerMetadata(random.randint(500,3000)))
        return ModelMetadata(layers, 0, 0, random.randint(0, 9))

    #Returns a ModelMetadata object representing a small mutation of its parents'
    #input metadata m
    #heaviness - allows manipulation of the extent of possible mutation. Values
    #of 1 to 3 are good?
    @staticmethod
    def mutate(input_m, heaviness):
        m = copy.deepcopy(input_m)
        index = random.randint(0, len(m.layers) - 1)
        for i in range(10 + heaviness):               
            if type(m.layers[index]) is ConvLayerMetadata:
                x = random.randint(0,2)
                if x == 0:
                    m.layers[index].data[0] = max(1, m.layers[index].data[0] + random.randint(-2, 3))
                if x == 1:
                    m.layers[index].data[1] = max(1, m.layers[index].data[1] + random.randint(-1, 1))
                if x == 2:
                    m.layers[index].data[2] = max(1, m.layers[index].data[2] + random.randint(-1, 1))
            if type(m.layers[index]) is DenseLayerMetadata:
                m.layers[index].data[0] = max(10, + m.layers[index].data[0] + random.randint(-5, 5))
        m.evolvedness += 1
        return m
    
    #Returns a metadata object formed by breeding the metadata elements of m_list
    #Because m_list can have a variable length, it's possible for an entire
    #population to contribute to one model's DNA
    @staticmethod
    def breed(m_list):
        m_list.sort(key=lambda metadata: len(metadata.layers))
        dense_chosen = False                      
        num_layers = random.randint(len(m_list[0].layers), len(m_list[-1].layers))
        layers = []
        for i in range(num_layers):
            while True:
                index = random.randint(0, len(m_list) - 1)
                if i < len(m_list[index].layers):
                    if type(m_list[index].layers[i]) is DenseLayerMetadata:
                        dense_chosen = True
                    if not (dense_chosen and type(m_list[index].layers[i]) is ConvLayerMetadata):
                        layers.append(m_list[index].layers[i])
                        break;
        evolvedness = 0
        for m in m_list:
            evolvedness += m.evolvedness
        evolvedness = evolvedness / len(m_list) + 1
        m = ModelMetadata(layers, evolvedness, 0, random.randint(0, 31))
        return m
        

In [0]:
global best_accuracies
best_accuracies = np.zeros(num_generations)

from sklearn.metrics import accuracy_score
#Returns the accuracy of the models, as an ensemble with weighted voting
def get_ensemble_accuracy(model_metadata):
    models = []
    for m in model_metadata:
        models.append(ModelWithMetadata(m))
    if len(models) == 0:
        print("The models went extinct")
        return 0.0
    models.sort(reverse=True, key=lambda model: model.metadata.accuracy) #Best model in smallest index
    results = np.array([model.model.predict_proba(test_images) for model in models])
    predictions_temp = np.zeros(shape=(len(test_images), 10))
    for i in range(results.shape[0]):
        for j in range(results.shape[1]):
            x = np.argmax(results[i][j])
            predictions_temp[j][x] += models[i].metadata.accuracy - models[-1].metadata.accuracy
    predictions = np.array([np.argmax(arr) for arr in predictions_temp])
    return accuracy_score(test_labels, predictions)

#Returns if the best accuracy hasn't increased recently enough, if so, it's
#experience shows that it's unlikely that it will soon
def evaluate_progress(generation):
    to_return = (False, False)
    try:
        if best_accuracies[generation] <= best_accuracies[generation-2]:
            to_return[0] = True
    except:
        pass
    try:
        if best_accuracies[generation] <= best_accuracies[generation-3]:
            to_return[1] = True
    except:
        pass
    return to_return
        

#Returns the average accuracy of a group of models
def get_generation_stats(model_metadata, generation):
    generation_accuracy = 0
    for m in model_metadata:
        generation_accuracy += m.accuracy
    best_accuracies[generation] = model_metadata[0].accuracy
    x = evaluate_progress(generation)
    return x[0], x[1], generation_accuracy / len(model_metadata)

#Assumes there are accuracies
def evolve(num_generations):
    counter = 0
    need_new_genes = False #Keeps track of a rough indicator of if there's too little diversity
    model_metadata = []
    best_metadata = []
    #Construct generation zero
    counter = 0
    while len(model_metadata) < 20:
        m = ModelWithMetadata(None)
        model_metadata.append(m.metadata)
        print(counter, "Added", m.metadata.to_string(), len(model_metadata))
        counter += 1
        del m
    print("Got initial models")
    for i in range(num_generations):
        counter = 0
        #Best model in smallest index
        model_metadata.sort(reverse=True, key=lambda metadata: metadata.accuracy) 
        #Get the accuracy of the generation
        results = get_generation_stats(model_metadata, i)
        print("GENERATION:", i, "Best accuracy:", model_metadata[0].accuracy, "Average accuracy:", results[2], "New genes needed:", results[0])
        print(model_metadata[0].to_string())

        print("\n")
        print("\n")

        #Add asexual models
        for i in range(min(2, len(model_metadata))):
            m = ModelWithMetadata(ModelWithMetadata.mutate(model_metadata[i], 1))
            model_metadata.append(m.metadata)
            print(counter, "Added", m.metadata.to_string())
            counter += 1
            del m
            m = ModelWithMetadata(ModelWithMetadata.mutate(model_metadata[i], 1))
            model_metadata.append(m.metadata)
            print(counter, "Added", m.metadata.to_string())
            counter += 1
            del m
                                  
        #Allow only good models to breed
        model_metadata.sort(reverse=True, key=lambda m: m.accuracy)
        
        done = False
        for m in model_metadata:
            if m.accuracy > .96:
                best_metadata.append(m)
                done = True
        if done:
            print("Ending generation early")
            break

        
        del model_metadata[10 - i:]
                
        #Add sexual models from best 3
        #for j in range(8):
            #rand_1 = random.randint(0, min(3, len(model_metadata) - 1))
            #rand_2 = random.randint(0, min(3, len(model_metadata) - 1))
            #rand_3 = random.randint(0, len(model_metadata) - 1)
            #rand_4 = random.randint(0, len(model_metadata) - 1)
            #models_to_breed = []
            #models_to_breed.append(model_metadata[rand_1])
            #models_to_breed.append(model_metadata[rand_2])
            #models_to_breed.append(model_metadata[rand_3])
            #models_to_breed.append(model_metadata[rand_4])
            #m = ModelWithMetadata(ModelWithMetadata.breed(models_to_breed))
            #model_metadata.append(m.metadata)
           # print(counter, "Added", m.metadata.to_string())
            #counter += 1
            #del m

    return best_metadata

In [0]:
#Returns the top three models from several bloodlines from which intermingling is
#forbidden to be used in a voting algorithm. This is to hopefully maximize the variance
#in the most accurate votes
#Inputs: num_generations per bloodline, num_bloodlines
def get_evolution_results(num_generations=8, bloodlines=8):
    best_models = []
    for i in range(bloodlines):
        returned_models = sorted(evolve(num_generations), reverse=True, key=lambda model: model.accuracy)
        del returned_models[3:]
        for model in returned_models:
            best_models.append(model)
    print("Final ensemble accuracy: ", get_accuracy(best_models))
    
get_evolution_results(5,8)

0 Added Dense 906 Dense 2112 Dense 1945 Dense 757 Dense 2416 Dense 1178  Evolvedness: 0 Accuracy:0.9243888888888889 1
1 Added Dense 1033 Dense 1408 Dense 1485 Dense 1991 Dense 2502 Dense 1331  Evolvedness: 0 Accuracy:0.8888333333333334 2
2 Added Dense 2376 Dense 1249 Dense 666 Dense 2496  Evolvedness: 0 Accuracy:0.9264444444444444 3
3 Added Dense 2202 Dense 1323 Dense 595 Dense 2694 Dense 2418 Dense 1393  Evolvedness: 0 Accuracy:0.9243333333333333 4
4 Added Dense 945 Dense 1532 Dense 1156 Dense 1671  Evolvedness: 0 Accuracy:0.9293888888888889 5
5 Added Dense 1786 Dense 567 Dense 1551  Evolvedness: 0 Accuracy:0.924 6
6 Added Dense 1295 Dense 1696 Dense 2061  Evolvedness: 0 Accuracy:0.9265 7
7 Added Dense 1448 Dense 1963 Dense 2489 Dense 990  Evolvedness: 0 Accuracy:0.9255555555555556 8
8 Added Dense 2376 Dense 2000 Dense 2486 Dense 2941 Dense 1648 Dense 2429  Evolvedness: 0 Accuracy:0.9042222222222223 9
9 Added Dense 723 Dense 1085 Dense 2342  Evolvedness: 0 Accuracy:0.9264444444444444 